In [6]:
import pandas as pd
import finder

In [7]:
def findAverageImageColor(image):
  r_sum = 0
  g_sum = 0
  b_sum = 0
  for x in range(len(image)):
    for y in range(len(image[0])):
      pixel = image[x][y]
      r_sum += pixel[0]
      g_sum += pixel[1]
      b_sum += pixel[2]
  count = len(image) * len(image[0])

  
  return {"avg_r": r_sum / count, "avg_g": g_sum / count, "avg_b": b_sum / count}

In [8]:
def createTilesDataFrame(path):
  data = []

  # Loop through all images in the `path` directory:
  for tileImageFileName in finder.listTileImagesInPath(path):
    # Load the image as a DataFrame and find the average color:
    image = finder.loadImage(tileImageFileName)
    averageColor = findAverageImageColor(image)

    # Store the fileName and average colors in a dictionary:
    d = { "fileName": tileImageFileName, "r": averageColor["avg_r"], "g": averageColor["avg_g"], "b": averageColor["avg_b"] }
    data.append(d)
    
  # Create the `df_tiles` DataFrame:
  df_tiles = pd.DataFrame(data)
  return df_tiles

In [9]:
def findAverageImageColorInBox(image, box_x, box_y, box_width, box_height):
  r_sum = 0
  g_sum = 0
  b_sum = 0
  for x in range(box_width):
    for y in range(box_height):
      pixel = image[box_x + x][box_y + y]
      r_sum += pixel[0]
      g_sum += pixel[1]
      b_sum += pixel[2]
  count = (box_width) * (box_height)

  return {"avg_r": r_sum / count, "avg_g": g_sum / count, "avg_b": b_sum / count}

In [10]:
import math

def findBestTile(df_tiles, r_avg, g_avg, b_avg):
  for i in df_tiles:
    dist = (((df_tiles["r"] - r_avg)**2) + ((df_tiles["g"] - g_avg)**2) + ((df_tiles["b"] - b_avg)**2))**.5
    df_tiles["dist"] = dist

  return df_tiles.nsmallest(1, "dist")

In [11]:
# What is your base image file name?
baseImageFileName = "my_cute_dog.jpeg"

# What folder contains your tile images?
# - You can change this so you can have multiple different folders of tile images.
tileImageFolder = "illinois-instagram-images"

# What is the maximum number of tiles should your mosaic use across?
# - More tiles across will increase the quality of the final image.
# - More tiles across will cause your program to run slower.
maximumTilesX = 200
# What height should your tiles be in your mosaic?
# - A larger tile image will result in a larger output file.
# - A larger tile image will result in your program running slower.
# - A larger tile image will result in more detail in the output file.
tileHeight = 32

In [12]:
print(f"Creating `df_tiles` from tile images in folder `{tileImageFolder}`...")
df_tiles = createTilesDataFrame(tileImageFolder)
print(f"...found {len(df_tiles)} tile images!")
df_tiles

Creating `df_tiles` from tile images in folder `illinois-instagram-images`...


FileNotFoundError: [Errno 2] No such file or directory: 'illinois-instagram-images/'

In [13]:
import sys
print(f"Loading your base image `{baseImageFileName}`...")
baseImage = finder.loadImage(baseImageFileName)
width = len(baseImage)
height = len(baseImage[0])


print(f"Finding best replacement image for each tile...")
# Find the pixelsPerTile to know the pixels used in the base image per mosaic tile:
import math

pixelsPerTile = int(math.ceil(width / maximumTilesX))
width = int(math.floor(width / pixelsPerTile) * pixelsPerTile)
height = int(math.floor(height / pixelsPerTile) * pixelsPerTile)
tilesX = int(width / pixelsPerTile)
tilesY = int(height / pixelsPerTile)

# Create the mosaic:
from PIL import Image
mosaic = Image.new('RGB', (int(tilesX * tileHeight), int(tilesY * tileHeight)))
for x in range(0, width, pixelsPerTile):
  for y in range(0, height, pixelsPerTile):
    avg_color = findAverageImageColorInBox(baseImage, x, y, pixelsPerTile, pixelsPerTile)
    replacement = findBestTile(df_tiles, avg_color["avg_r"], avg_color["avg_g"], avg_color["avg_b"])

    tile = finder.getTileImage(replacement["fileName"].values[0], tileHeight)
    mosaic.paste(tile, (int(x / pixelsPerTile) * tileHeight, int(y / pixelsPerTile) * tileHeight))

  # Print out a progress message:
  curRow = int((x / pixelsPerTile) + 1)
  pct = (curRow / tilesX) * 100
  sys.stdout.write(f'\r  ...progress: {curRow * tilesY} / {tilesX * tilesY} ({pct:.2f}%)')

# Save it
mosaic.save('mosaic-hd.jpg')

# Save a smaller one (for posting):
import PIL
d = max(width, height)
factor = d / 4000
if factor <= 1: factor = 1

small_w = width / factor
small_h = height / factor    
baseImage = mosaic.resize( (int(small_w), int(small_h)), resample=PIL.Image.LANCZOS )
baseImage.save('mosaic-web.jpg')

print("")
print("")
print(f"MOSAIC COMPLETE!")
print("- See `mosaic-hq.jpg` to see your HQ moasic! (The file may be HUGE.)")
print("- See `mosaic.jpg` to see a moasic best suited for the web (still big, but not HUGE)!")

Loading your base image `my_cute_dog.jpeg`...
Finding best replacement image for each tile...


NameError: name 'df_tiles' is not defined